<a href="https://colab.research.google.com/github/Sibaso/Project2_quan.nt173312/blob/master/Neuron_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuron Network
Lấy dữ liệu ở file ở dạng ma trận thưa
Chuyển dữ liệu sang kiểu tensor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch

def load_data(data_path, vocab_size):
  with open(data_path, encoding = 'latin1') as f:
    d_lines = f.read().splitlines()
  data,labels = [],[]
  for data_id, line in enumerate(d_lines):
    vector = [0.0 for _ in range(vocab_size)]
    features = line.split('<fff>')
    label, doc_id = int(features[0]), int(features[1])
    for token in features[2].split():
      index, value = int(token.split(':')[0]), float(token.split(':')[1])
      vector[index] = value
    data.append(vector)
    labels.append(label)
  return torch.tensor(data), torch.tensor(labels)

with open('/content/drive/My Drive/Data_Colab/words_idf.txt', encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())
X_train, Y_train = load_data(
    '/content/drive/My Drive/Data_Colab/train_tf_idf_vector.txt', vocab_size)
X_test,Y_test = load_data(
    '/content/drive/My Drive/Data_Colab/test_tf_idf_vector.txt', vocab_size)

Shuffle dữ liệu rồi tách ra thành các batch, mỗi lần sẽ thực hiện forward và backward cho toàn bộ phần tử trong batch

In [0]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train, Y_train)
train_data_loader = DataLoader(train_dataset, batch_size = 50, shuffle = True)

- Xây dựng các tham số cho mạng sử dụng torch.nn.Linear gồm các weight và bias cho từng tầng
- Lan truyền tiến (forward): tầng ẩn sử dụng hàm kích hoạt là sigmoid(x) = 1/(1+exp(-x)), tầng đầu ra sử dụng hàm softmax(xi) = exp(xi)/∑j exp(xj)
- Tính lỗi dựa trên đầu ra của lan truyền tiến và nhãn thực tế của ví dụ sử dụng hàm cross_entropy
- Sử dụng torch.optim.Adam để cập nhật tham số dựa trên lỗi tính được

In [0]:
import torch.nn.functional as F

class NeuronNetwork(torch.nn.Module):
  def __init__(self, vocab_size, hidden_size, num_classes):
    super().__init__()
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
    self._num_classes = num_classes

  #xay dung cau truc mang
  def build_graph(self):
    self._hidden_layer = torch.nn.Linear(self._vocab_size, self._hidden_size)
    self._output_layer = torch.nn.Linear(self._hidden_size, self._num_classes)

  def forward(self,x):
    x = torch.sigmoid(self._hidden_layer(x))
    x = F.softmax(self._output_layer(x), dim = 1)
    return x

  def fit(self, max_epochs, data_loader, learning_rate, threshold):
    self.build_graph()
    opt = torch.optim.Adam(params = self.parameters(), lr = learning_rate)
    last_loss = 0
    for epoch in range(max_epochs):
      new_loss = 0
      for data,labels in data_loader:
        self.zero_grad()
        #lan truyen tien, tinh dau ra
        prediced = self.forward(data)
        #xac dinh loi
        loss = F.cross_entropy(prediced, labels)
        new_loss += loss
        #lan truyen nguoc
        loss.backward()
        #cap nhat tham so
        opt.step()
      new_loss = new_loss / len(data_loader)
      print('round: {}, loss: {}'.format(epoch, new_loss))
      if abs(last_loss - new_loss) <= threshold:
        return
      last_loss=new_loss

  def predict(self, X):
    return torch.argmax(self.forward(X), dim = 1)

  def compute_accuracy(self,predicted,expected):
    return (predicted == expected).float().mean()


Huấn luyện mạng neuron và tính độ chính xác trên tập huấn luyện với tập thử nghiệm

In [6]:
NN = NeuronNetwork(
    vocab_size = vocab_size,
    hidden_size = 50,
    num_classes = 20
    )
NN.fit(
    max_epochs = 20,
    data_loader = train_data_loader,
    learning_rate = 1e-2,
    threshold = 1e-3
    )
predicted = NN.predict(X_train)
print('train accuracy:', NN.compute_accuracy(predicted, Y_train))
predicted = NN.predict(X_test)
print('test accuracy:', NN.compute_accuracy(predicted, Y_test))

round: 0, loss: 2.750535488128662
round: 1, loss: 2.2983288764953613
round: 2, loss: 2.1613996028900146
round: 3, loss: 2.128693103790283
round: 4, loss: 2.1092355251312256
round: 5, loss: 2.0916218757629395
round: 6, loss: 2.086991310119629
round: 7, loss: 2.084772825241089
round: 8, loss: 2.083759307861328
round: 9, loss: 2.083280563354492
train accuracy: tensor(0.9958)
test accuracy: tensor(0.8411)


Các tham số của mạng sau khi huấn luyện

In [7]:
print('hidden layer weight:', NN._hidden_layer.weight.shape)
print(NN._hidden_layer.weight)
print('hidden layer bias:', NN._hidden_layer.bias.shape)
print(NN._hidden_layer.bias)
print('output layer weight:', NN._output_layer.weight.shape)
print(NN._output_layer.weight)
print('output layer bias:', NN._output_layer.bias.shape)
print(NN._output_layer.bias)

hidden layer weight: torch.Size([50, 14612])
Parameter containing:
tensor([[ 0.0582,  0.2224, -0.9582,  ...,  0.7449,  0.6211, -0.0066],
        [ 0.4620,  0.3086, -0.3577,  ..., -0.0907, -0.4680,  0.0040],
        [ 0.2072,  0.7601,  0.1113,  ...,  0.0922, -0.0977,  0.0078],
        ...,
        [-0.1153,  0.0858, -0.1707,  ..., -0.2093,  0.1613, -0.0064],
        [-0.3302, -0.4415,  0.0305,  ..., -0.0348, -0.5109,  0.0026],
        [ 0.4458, -0.2334,  0.3221,  ...,  0.1679,  0.0893, -0.0076]],
       requires_grad=True)
hidden layer bias: torch.Size([50])
Parameter containing:
tensor([ 0.2047,  0.1319,  0.1494, -0.0049,  0.0114,  0.1514,  0.2834,  0.1989,
         0.2642,  0.1540,  0.0760,  0.0251,  0.1785, -0.0126, -0.0044, -0.1902,
        -0.2549, -0.1546,  0.3797,  0.0419, -0.0953,  0.2001,  0.1680,  0.2071,
         0.2524,  0.2435,  0.0105,  0.2383,  0.3663,  0.0298, -0.0239,  0.1038,
        -0.2932, -0.0515,  0.1688,  0.1071,  0.3186,  0.2966,  0.0319,  0.0960,
         0.114

# SVMs

In [15]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.svm import SVC

def compute_accuracy(predicted,expected):
	matches=np.equal(predicted,expected)
	accuracy=np.sum(matches)/len(expected)
	return accuracy

train_data = csr_matrix(X_train)
test_data = csr_matrix(X_test)

classifier=SVC(
		C=50,
		kernel='rbf',
		gamma=0.1,
		tol=1e-3,
		verbose=True
    )
classifier.fit(train_data,np.array(Y_train))
predicted=classifier.predict(test_data)
accuracy=compute_accuracy(predicted=predicted,expected=np.array(Y_test))
print('accuracy:',accuracy)

[LibSVM]accuracy: 0.8250132766861391
